In [229]:
import re
import pandas as pd
from bertopic import BERTopic

In [248]:
# df_candidate_tweets = pd.read_csv('../data/twitter_data/jadot.csv', encoding='utf-8')
# # df_candidate_tweets.dropna(inplace=True)
# df_candidate_tweets.shape

(1157, 14)

In [230]:
import os
files = os.listdir('../data/twitter_data/')
df_candidate_tweets = pd.DataFrame()
for f in files:
    df = pd.read_csv(f'../data/twitter_data/{f}', encoding='utf-8')
    df_candidate_tweets = df_candidate_tweets.append(df)

print(df_candidate_tweets.shape)

(7676, 14)


In [231]:
# preparing tweet texts
import preprocessing as pr

df_candidate_tweets['preprocessed_tweet'] = df_candidate_tweets.tweet_text.apply(pr.preprocess_tweet)
df_candidate_tweets['basic_cleaning'] = df_candidate_tweets.tweet_text.apply(pr.basic_clean) # pr.basic_clean

df_candidate_tweets.basic_cleaning = df_candidate_tweets.basic_cleaning.astype('string')
df_candidate_tweets.basic_cleaning = df_candidate_tweets.apply(lambda row: re.sub(r"http\S+", "", row.basic_cleaning).lower(), 1)
df_candidate_tweets.basic_cleaning = df_candidate_tweets.apply(lambda row: " ".join(filter(lambda x:x[0]!="@", row.basic_cleaning.split())), 1)
df_candidate_tweets.basic_cleaning = df_candidate_tweets.apply(lambda row: " ".join(re.sub("[^a-zA-Z]+", " ", row.basic_cleaning).split()), 1)

print(df_candidate_tweets.shape)
# len(df_candidate_tweets)

(7676, 16)


In [232]:
# filtering date
df_candidate_tweets.time = pd.to_datetime(df_candidate_tweets['time'], dayfirst=True)
start_date = '2021-09-01'
end_date = '2021-10-30'
mask = (df_candidate_tweets['time'] > start_date) & (df_candidate_tweets['time'] <= end_date)
df_candidate_tweets_in_timeframe = df_candidate_tweets.loc[mask]
print(df_candidate_tweets_in_timeframe.shape)

(6463, 16)


In [233]:
# saving text and time in 2 different lists
timestamps = df_candidate_tweets_in_timeframe.time.to_list()
tweets = df_candidate_tweets_in_timeframe.preprocessed_tweet.to_list()

In [235]:
# getting topics
topic_model = BERTopic(language="multilingual")
topics, probabilities = topic_model.fit_transform(tweets)

In [238]:
topic_model.get_topic_freq().head(50)

,Topic,Count
0,-1,2410
1,0,558
2,1,176
3,2,169
4,3,134
5,4,122
6,5,117
7,6,113
8,7,112
9,8,112


In [247]:
topic_model.get_topic(19)

[('fiscale', 0.08204027550022916),
 ('fraude', 0.07488825725651986),
 ('fiscaux', 0.029108407203799123),
 ('argent', 0.028289280656158638),
 ('impôt', 0.02765367189849931),
 ('scandale', 0.021331948059918043),
 ('financier', 0.01739892848196386),
 ('tax', 0.01630454347094405),
 ('corrompus', 0.014387641717251985),
 ('fraudeurs', 0.013657484172958125)]

In [240]:
topic_model.visualize_topics()

In [241]:
topic_model.visualize_barchart()

In [243]:
topic_model.visualize_heatmap()

In [ ]:
# finding topic's distribbution over time
topics_over_time = topic_model.topics_over_time(tweets, topics, timestamps, nr_bins=20)

In [ ]:
# visualization
topic_model.visualize_topics_over_time(topics_over_time, top_n_topics=10)

In [ ]:
print(topics_over_time.sort_values('Frequency', ascending=False, na_position='first'))